In [35]:
import ipywidgets as pw
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
no_points = 200
x = np.linspace(0, 10, no_points)
y = np.linspace(0, 10, no_points)
x, y = np.meshgrid(x, y)
class onda():
  def __init__(self, x, y):
    # self.x = x
    # self.y = y
    self.Lx = x.max()
    self.Ly = y.max()
    self._set_widget_controls()
  def _get_widgets(self):
    self.nx = pw.IntSlider(min = 1, max = 10, step = 1 , value = 1, description = 'nx')
    self.ny = pw.IntSlider(min = 1, max = 10, step = 1 , value = 1, description = 'ny')
    self.Amplitud = pw.FloatSlider(min = -1, max = 1, step = .25, value = 0, description = 'Amplitud')
    return {'nx' : self.nx, 'ny': self.ny, 'Amplitud' : self.Amplitud}
  def __call__(self, x, y, t, tension=1, masa=1) :
    kx = self.nx.value*np.pi/self.Lx
    ky = self.ny.value*np.pi/self.Ly
    k = (kx**2+ky**2)**.5
    v = (tension/masa)**.5
    omega = v*k
    self.frecuencia = omega/(2*np.pi)
    return self.Amplitud.value*np.cos(omega*t/10)*np.sin(kx*x)*np.sin(ky*y)
  def _set_widget_controls(self):
      self.controles = self._get_widgets()
      self.panel_control_widget = pw.VBox([key for val, key in self.controles.items()])

# fig, ax = plt.subplots()
# display((pw.interactive(w1._funcion_interactiva(x, 0, ax), w1.controles)).children[-1], w1.menu)
MAX_NO_ONDAS = 10
ondas = [onda(x, y) for n in range(MAX_NO_ONDAS)]
ondas[0].Amplitud.set_trait('value',1)
No_ondas = pw.IntSlider(min = 1, max = MAX_NO_ONDAS, value = 5, step = 1)
cmap = pw.Select( value = 'jet', options = ['jet', 'gray', 'hot', 'bwr'])
def plot_ondas(No_ondas, cmap = 'jet'):
    surf = 0*x
    acordeon = pw.Accordion(children= [w.panel_control_widget for w in ondas[:No_ondas]], titles = [f'Onda #{index+1}' for index in range(No_ondas)])
    Tiempo = pw.Play(value=0,min=0,max=10000,step=2,interval=250,disabled=False)
    slider = pw.IntSlider(description = 'Tiempo')
    pw.jslink((Tiempo, 'value'), (slider, 'value'))
    # pw.HBox([play, slider])
    output = pw.interactive_output(lambda Tiempo : (plt.pcolormesh(x, y, sum([w(x, y, Tiempo) for w in ondas[:No_ondas]]), vmax =1, vmin = -1, cmap = cmap), plt.annotate('\n'.join([f'$F_{{ {index+1} }} = {np.round(w.frecuencia, 2)}Hz$' for index, w in enumerate(ondas[:No_ondas])]), xy = (10,0)), plt.show()), {'Tiempo': Tiempo})
    control = pw.HBox([acordeon,Tiempo])
    display(slider, pw.HBox([output, control]))
if __name__ == '__main__':
    pw.interact(plot_ondas, No_ondas = No_ondas, cmap = cmap)


interactive(children=(IntSlider(value=5, description='No_ondas', max=10, min=1), Select(description='cmap', op…